In [1]:
import numpy as np         # библиотека для матриц и математики
import pandas as pd        # библиотека для работы с табличками
from scipy import stats    # модуль для работы со статистикой

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

import math

from statsmodels.stats.weightstats import ztest as ztest

plt.style.use('ggplot')    # стиль графиков

%matplotlib inline


In [2]:
df = pd.read_excel(r"C:\Users\Пользователь\Downloads\Telegram Desktop\data_matstat_K5.xls", sheet_name="B - bodyfat")

In [17]:
std1 = df['B10'].std()
std2 = df['B11'].std()
mean1 = df.B10.mean()
mean2 = df.B11.mean()
n = df.B11.shape[0]

sample1 = df['B10'].to_numpy()
sample2 = df['B11'].to_numpy()

s = 0
for i in range(0, n):
  s += (sample1[i] - mean1) * (sample2[i] - mean2)
k = s / n

r = k / (std1 * std2)

alpha = 0.1

quantile = stats.norm(loc = 0, scale = 1).ppf(1 - (alpha/2))

a = r + (r*(1-r**2)/(2*n)) - ((1-r**2)/n**(1/2)) * quantile
b = r + (r*(1-r**2)/(2*n)) + ((1-r**2)/n**(1/2)) * quantile

print(a)
print(b)


0.8950998638613753
0.9298947806201897


In [5]:
n

252

In [14]:
std1 = df['B10'].std()
std2 = df['B11'].std()
mean1 = df.B10.mean()
mean2 = df.B11.mean()
n = df.B11.shape[0]

sample1 = df['B10'].to_numpy()
sample2 = df['B11'].to_numpy()

s = 0
for i in range(0, n):
  s += (sample1[i] - mean1) * (sample2[i] - mean2)
k = s / n

r = k / (std1 * std2)

alpha = 0.01

z = (r / (1 - r**2)**(1/2)) * (n-2)**(1/2)
f_z = stats.t(n-2).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test1: z = %.3f; p-value = %.3f" % (z, p_value))

test1: z = 35.199; p-value = 0.000


In [7]:
s1 = pd.Series(sample1)
s2 = pd.Series(sample2)


sp = 0
for i in range(0, n):
  sp += (s1.rank()[i] - s2.rank()[i])**2
sp = 1 - (6 / (n * (n**2 - 1))) * sp

z = (sp / (1 - sp**2)**(1/2)) * (n-2)**(1/2)

f_z = stats.t(n-2).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test2: z = %.3f; p-value = %.3f" % (z, p_value))

test2: z = 31.771; p-value = 0.000


In [8]:
ken = stats.kendalltau(sample1, sample2)

z = ken[0] * ((9*n*(n+1))/(2*(2*n+5)))**(1/2)
f_z = stats.norm(loc = 1, scale = 1).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test3: z = %.3f; p-value = %.3f" % (z, p_value))

test3: z = 17.196; p-value = 0.000


In [9]:
#KK

r = stats.pearsonr(sample1, sample2)
p = stats.spearmanr(sample1, sample2)   #спирмен
t = stats.kendalltau(sample1, sample2)  #кендалл
print(r)
print(p)
print(t)
type(t[0])

PearsonRResult(statistic=0.9158276698546782, pvalue=4.7986071277865885e-101)
SignificanceResult(statistic=0.8952563412689127, pvalue=9.432456663616326e-90)
SignificanceResult(statistic=0.7243113600801906, pvalue=2.2625919542765998e-65)


numpy.float64

In [11]:


sample1 = df['B10'].to_numpy()
sample2 = df['B11'].to_numpy()
sample3 = df['B12'].to_numpy()

n1 = len(sample1)
n2 = len(sample2)
n3 = len(sample3)

In [20]:
# матрица ранговых коэффициентов корреляции по Кендаллу
# матрица значений p-value для ранговых коэффициентов корреляции по Кендаллу

sample = [sample1, sample2, sample3]
arr_kendall = []
arr_kendall_pvalue = []

for i in range(0,3):
  row = []
  row_p = []
  for j in range(0,3):
    if i != j:
      t = stats.kendalltau(sample[i], sample[j])
      row.append(t[0])
      row_p.append(t[1])
    else:
      t = stats.kendalltau(sample[i], sample[j])
      row.append(t[0])
      row_p.append('Null')
  arr_kendall.append(row)
  arr_kendall_pvalue.append(row_p)

arr_kendall_pvalue

[['Null', 2.2625919542765998e-65, 6.3554737796748166e-49],
 [2.2625919542765998e-65, 'Null', 8.526790929548107e-55],
 [6.3554737796748166e-49, 8.526790929548107e-55, 'Null']]

In [21]:
#точечная оценка коэффициента конкордации

s1 = pd.Series(sample1)
s2 = pd.Series(sample2)
s3 = pd.Series(sample3)

r1 = s1.rank()
r2 = s2.rank()
r3 = s3.rank()

r = [r1, r2, r3]

n = n1
k = 3       #число выборок

w = 0
for i in range(0, n):
  s1 = 0
  for j in range(0, k):
    s1 += (1/k) * r[j][i]
  w += (s1 - (n+1)/2)**2

w = (w * 12) / (n**3 - n)

print('Коэффициент конкордации: %.3f' % w)

#статистика критерия

z = k * (n - 1) * w
f_z = stats.chi2(n-1).cdf(z)
p_value = 2 * min(f_z, 1 - f_z)

print("test: z = %.3f; p-value = %.3f" % (z, p_value))

Коэффициент конкордации: 0.900
test: z = 678.045; p-value = 0.000
